In [1]:
import os
import tempfile

%matplotlib inline
import matplotlib.pyplot as plt

import numpy as np
import tensorflow as tf
import tensorflow_datasets as tfds

import tensorflow_recommenders as tfrs

plt.style.use('seaborn-whitegrid')

2023-03-24 13:36:03.305554: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/spark/miniconda3/envs/recommend/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/tmp/ipykernel_365585/2468769046.py:13: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn-whitegrid')


In [2]:
from pymongo import MongoClient
import pandas as pd

mongo = MongoClient()
data = list(mongo["data_3"]["events"].find({},{"_id": 0}))
movies = list(mongo["data_3"]["data"].find({},{"_id": 0, "tag": 0}))
df_meta = pd.DataFrame(movies)
df = pd.DataFrame(data)

BEHAVIORS = {
    "apply": 5.0,
    "search": 4.0,
    "like": 4.0,
    "view": 3.0,
    "ignore": 1.0,
    "dislike": 0.0,
    "hated": 0.0,
}

def create_rating(behavior):
    return BEHAVIORS.get(behavior, 0)
    
df["user_rating"] = df.behavior.apply(create_rating)
ratings = df[["userid", "jobId", "location", "level","behavior"]]

tensor_slices = {
    "user_id": ratings.userid.values.tolist(),
    "movie_title": ratings.jobId.values.tolist(),
    "location": ratings.location.values.tolist(),
    "level": ratings.level.values.tolist(),
    "behavior": ratings.behavior.values.tolist(),
    
}

movies = tf.data.Dataset.from_tensor_slices({
    "movie_title": df_meta.jobId.values.tolist(),
    "location": df_meta.location.values.tolist(),
    "level": df_meta.level.values.tolist(),
})
ratings = tf.data.Dataset.from_tensor_slices(tensor_slices)

2023-03-24 13:36:04.965222: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 13:36:04.986131: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 13:36:04.986293: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-03-24 13:36:04.986685: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorF

In [3]:
unique_movie_titles = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["movie_title"]))))
unique_user_ids = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["user_id"]))))
unique_locations = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["location"]))))
unique_levels = np.unique(np.concatenate(list(ratings.batch(1_000).map(
    lambda x: x["level"]))))

Instructions for updating:
Lambda fuctions will be no more assumed to be used in the statement where they are used, or at least in the same block. https://github.com/tensorflow/tensorflow/issues/56089


In [4]:
class UserModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    self.user_embedding = tf.keras.Sequential([
        tf.keras.layers.StringLookup(
            vocabulary=unique_user_ids, mask_token=None),
        tf.keras.layers.Embedding(len(unique_user_ids) + 1, 32),
    ])

  def call(self, inputs):
    # Take the input dictionary, pass it through each input layer,
    # and concatenate the result.
    return tf.concat([
        self.user_embedding(inputs["user_id"])
    ], axis=1)

In [5]:
UserModel()({"user_id": np.array(["User_Senior_Devops_Engineer_other_37"]),
           "behavior": np.array(["apply"]),
          })

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[-0.02636201,  0.00274365, -0.04057433, -0.00938762, -0.03017684,
         0.04830516, -0.03963566,  0.03766796, -0.01012691,  0.00378395,
         0.00423228,  0.00362388, -0.00556765, -0.04892868,  0.03063529,
         0.01848949,  0.04872996,  0.00071552,  0.01621049,  0.01758719,
         0.02100046,  0.04453533,  0.00591059, -0.00662664, -0.00925561,
         0.01364339, -0.03555019, -0.04940001,  0.03272131,  0.02823527,
        -0.02054337,  0.04104196]], dtype=float32)>

In [6]:
class QueryModel(tf.keras.Model):
  """Model for encoding user queries."""

  def __init__(self, layer_sizes):
    """Model for encoding user queries.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    # We first use the user model for generating embeddings.
    self.embedding_model = UserModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    print(feature_embedding)
    return self.dense_layers(feature_embedding)

In [7]:
QueryModel([32])({"user_id": np.array(["User_Senior_Devops_Engineer_other_37"]),
           "behavior": np.array(["apply"]),
          })

tf.Tensor(
[[-0.0304952   0.03654505  0.00927611 -0.00733347 -0.02918608 -0.02933254
  -0.04439579 -0.04627618  0.01602227 -0.01628242 -0.0367323   0.00274659
   0.01977886 -0.01454793  0.01396606  0.04578543 -0.01580743 -0.01700627
  -0.02394766  0.02622343 -0.04480838 -0.02986836 -0.02698712 -0.01333393
  -0.04459727 -0.04979414  0.00419306  0.0053679  -0.00923055 -0.03620316
   0.04487218 -0.03693352]], shape=(1, 32), dtype=float32)


<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ 0.01477079,  0.03042125, -0.03349719, -0.04650142,  0.00963766,
         0.04332083, -0.00033333,  0.02700805,  0.02842907,  0.02399847,
         0.02143316, -0.01936558,  0.01489903,  0.0614911 ,  0.00662774,
         0.00815285,  0.10748338,  0.00971411, -0.02992071,  0.03018531,
        -0.00824842,  0.01304959,  0.00978306,  0.00409078, -0.05776342,
         0.01213767,  0.07155527, -0.00908046, -0.0125214 ,  0.00249023,
        -0.03918808, -0.02678093]], dtype=float32)>

In [8]:
class MovieModel(tf.keras.Model):

  def __init__(self):
    super().__init__()

    max_tokens = 10_000

    self.title_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_movie_titles,mask_token=None),
      tf.keras.layers.Embedding(len(unique_movie_titles) + 1, 32)
    ])
    self.location_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_locations,mask_token=None),
      tf.keras.layers.Embedding(len(unique_locations) + 1, 32)
    ])
    self.level_embedding = tf.keras.Sequential([
      tf.keras.layers.StringLookup(
          vocabulary=unique_levels,mask_token=None),
      tf.keras.layers.Embedding(len(unique_levels) + 1, 32)
    ])
#     self.behavior_embedding = tf.keras.Sequential([
#       tf.keras.layers.StringLookup(
#           vocabulary=np.array(["apply", "search", "view"]),
#           mask_token=None,
#           output_mode="tf-idf",
#           idf_weights=np.array([0.5, 0.3, 0.2])
#       ),
#       tf.keras.layers.Embedding(4, 32)
#     ])

  def call(self, features):
    return tf.concat([
        self.title_embedding(features["movie_title"]),
        self.location_embedding(features["location"]),
        self.level_embedding(features["level"]),
    ], axis=1)

In [9]:
MovieModel()({
        "user_id": np.array(["User_Senior_Devops_Engineer_other_37"]),
        "movie_title": np.array(["User_Senior_Devops_Engineer_other_37"]),
        "location": np.array(["HCM"]),
        "level": np.array(["Senior"]),
        "behavior": np.array(["apply"])
})

<tf.Tensor: shape=(1, 96), dtype=float32, numpy=
array([[ 0.01625573, -0.04908146,  0.03397213,  0.00876946, -0.01378919,
        -0.01384919,  0.02398394,  0.04642553, -0.01187866,  0.02905214,
         0.03261078, -0.00728115, -0.01702945, -0.01846369, -0.00106131,
        -0.03922448, -0.01425742,  0.04962252,  0.01736548, -0.02476326,
         0.00375404, -0.04261242,  0.04796085,  0.01912382,  0.01708921,
         0.02204614,  0.02071536,  0.02269968, -0.03066013, -0.00217479,
        -0.03591643,  0.01431955,  0.03797034, -0.00192641, -0.04887431,
        -0.04639303, -0.0373199 ,  0.01452402,  0.00087769, -0.01875049,
        -0.00431637,  0.02099235,  0.01487173, -0.04855778, -0.0036671 ,
        -0.03070921,  0.01316391, -0.02046546,  0.04964476,  0.00684429,
        -0.00821998,  0.0126976 ,  0.03192273,  0.01318209,  0.00153091,
         0.0233644 ,  0.01580117, -0.03011973,  0.01512108,  0.01607846,
         0.00378667,  0.01188957,  0.02622752, -0.02351552,  0.02061256,
  

In [10]:
class CandidateModel(tf.keras.Model):
  """Model for encoding movies."""

  def __init__(self, layer_sizes):
    """Model for encoding movies.

    Args:
      layer_sizes:
        A list of integers where the i-th entry represents the number of units
        the i-th layer contains.
    """
    super().__init__()

    self.embedding_model = MovieModel()

    # Then construct the layers.
    self.dense_layers = tf.keras.Sequential()

    # Use the ReLU activation for all but the last layer.
    for layer_size in layer_sizes[:-1]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size, activation="relu"))

    # No activation for the last layer.
    for layer_size in layer_sizes[-1:]:
      self.dense_layers.add(tf.keras.layers.Dense(layer_size))

  def call(self, inputs):
    feature_embedding = self.embedding_model(inputs)
    return self.dense_layers(feature_embedding)

In [11]:
CandidateModel([32])({
        "user_id": np.array(["User_Senior_Devops_Engineer_other_37"]),
        "movie_title": np.array(["User_Senior_Devops_Engineer_other_37"]),
        "location": np.array(["HCM"]),
        "level": np.array(["Senior"]),
        "behavior": np.array(["apply"])
})

<tf.Tensor: shape=(1, 32), dtype=float32, numpy=
array([[ 0.00815218, -0.05207117, -0.0066966 ,  0.02759909,  0.00991655,
         0.02498347,  0.02764549, -0.02515319,  0.00629235, -0.00372388,
        -0.00352932,  0.02859033, -0.04699264,  0.01495095, -0.04039677,
         0.02412187, -0.00079277,  0.05577357,  0.03379545, -0.00145828,
        -0.09511828,  0.06292261,  0.10407332,  0.01945008, -0.06183894,
        -0.04345604, -0.03639319,  0.01550556,  0.02002034,  0.03659528,
        -0.04495645,  0.06094392]], dtype=float32)>

In [12]:
class MovielensModel(tfrs.models.Model):

  def __init__(self, layer_sizes):
    super().__init__()
    self.query_model = QueryModel(layer_sizes)
    self.candidate_model = CandidateModel(layer_sizes)
    self.task = tfrs.tasks.Retrieval(
        metrics=tfrs.metrics.FactorizedTopK(
            candidates=movies.batch(128).map(self.candidate_model),
        ),
    )

  def compute_loss(self, features, training=False):
    # We only pass the user id and timestamp features into the query model. This
    # is to ensure that the training inputs would have the same keys as the
    # query inputs. Otherwise the discrepancy in input structure would cause an
    # error when loading the query model after saving it.
    query_embeddings = self.query_model({
        "user_id": features["user_id"]
    })
    movie_embeddings = self.candidate_model({
        "movie_title": features["movie_title"],
        "location": features["location"],
        "level": features["level"]
    })

    return self.task(
        query_embeddings, movie_embeddings, compute_metrics=not training)

In [13]:
tf.random.set_seed(42)
shuffled = ratings.shuffle(100_000, seed=42, reshuffle_each_iteration=False)

train = shuffled.take(80_000)
test = shuffled.skip(80_000).take(20_000)

cached_train = train.shuffle(100_000).batch(2048)
cached_test = test.batch(4096).cache()

In [14]:
num_epochs = 300

model = MovielensModel([256, 128, 64])
model.compile(optimizer=tf.keras.optimizers.Adagrad(0.1))

model.fit(
    cached_train,
    validation_data=cached_test,
    validation_freq=5,
    epochs=num_epochs,
    verbose=0)

Tensor("query_model_1/user_model_2/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("query_model_1/user_model_2/concat/concat:0", shape=(None, 32), dtype=float32)


2023-03-24 13:36:07.897815: I tensorflow/compiler/xla/service/service.cc:173] XLA service 0x7f545000e410 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-03-24 13:36:07.897833: I tensorflow/compiler/xla/service/service.cc:181]   StreamExecutor device (0): NVIDIA GeForce RTX 2070, Compute Capability 7.5
2023-03-24 13:36:07.900726: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2023-03-24 13:36:07.942262: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:07.967819: I tensorflow/compiler/jit/xla_compilation_cache.cc:477] Compiled clust

2023-03-24 13:36:09.860699: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:09.934234: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:10.010387: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-24 13:36:11.843301: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:11.921693: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:12.010334: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-24 13:36:14.259992: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:14.369691: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:14.541523: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-24 13:36:17.346844: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:17.541322: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:17.664028: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-24 13:36:23.151130: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:23.299205: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:23.535803: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

2023-03-24 13:36:41.960140: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:46.053539: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-address errors.

You may not need to update to CUDA 11.1; cherry-picking the ptxas binary is often sufficient.
2023-03-24 13:36:48.945968: W tensorflow/compiler/xla/stream_executor/gpu/asm_compiler.cc:115] *** WARNING *** You are using ptxas 10.1.243, which is older than 11.1. ptxas before 11.1 is known to miscompile XLA code, leading to incorrect results or invalid-addr

In [15]:
index = tfrs.layers.factorized_top_k.BruteForce(model.query_model, k=20)
index.index_from_dataset(
    movies.batch(100).map(lambda x: (x["movie_title"], model.candidate_model(x))))

In [16]:
_, pred = index({"user_id": np.array(["User_Fresher_NodeJS_Developer_DN_15"])})
pred.numpy()

tf.Tensor(
[[-0.19368547  0.16815117 -0.03889159  0.0110692   0.16332166  0.04609352
   0.18925816 -0.09922071 -0.04682924 -0.01661898 -0.05269184  0.06500857
   0.16101582 -0.13959606  0.04343922  0.19310878 -0.4364676   0.14631559
  -0.04928181 -0.00280597  0.10534646  0.2784824  -0.09638617 -0.07081282
   0.14778797 -0.01146519 -0.16668493 -0.2984882   0.04190021 -0.07911184
   0.04319108 -0.00234668]], shape=(1, 32), dtype=float32)


array([[b'Job_Fresher_NodeJS_Developer_DN_Fresher_GBKWPB',
        b'Job_Fresher_NodeJS_Developer_DN_Fresher_XQDV2R',
        b'Job_Fresher_NodeJS_Developer_other_Fresher_PJZZPQ',
        b'Job_Fresher_Fullstack_Developer_other_Fresher_WM5YTT',
        b'Job_Fresher_NodeJS_Developer_other_Fresher_8T0RDK',
        b'Job_Junior_NodeJS_Developer_DN_Junior_2QWO3K',
        b'Job_Fresher_NodeJS_Developer_other_Fresher_RH8E05',
        b'Job_Fresher_NodeJS_Developer_HN_Fresher_RNJIF0',
        b'Job_Fresher_NodeJS_Developer_other_Fresher_B1F3LE',
        b'Job_Fresher_NodeJS_Developer_other_Fresher_LCNO13',
        b'Job_Junior_Fullstack_Developer_DN_Junior_4AJ6YX',
        b'Job_Fresher_Fullstack_Developer_other_Fresher_I52ZGT',
        b'Job_Fresher_Fullstack_Developer_other_Fresher_ZJALU0',
        b'Job_Fresher_Fullstack_Developer_other_Fresher_GB1G57',
        b'Job_Fresher_Fullstack_Developer_other_Fresher_OE2NTE',
        b'Job_Fresher_Devops_Engineer_other_Fresher_DWH3YU',
        b'

In [17]:
path = "/home/spark/ylv/workplace/model"
tf.saved_model.save(
  index,
  path,
  options=tf.saved_model.SaveOptions(namespace_whitelist=["brute_force"])
)

Tensor("brute_force/query_model_1/user_model_2/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("user_model_2/StatefulPartitionedCall:0", shape=(None, 32), dtype=float32)
Tensor("user_model_2/StatefulPartitionedCall:0", shape=(None, 32), dtype=float32)
Tensor("user_model_2/StatefulPartitionedCall:0", shape=(None, 32), dtype=float32)
Tensor("user_model_2/StatefulPartitionedCall:0", shape=(None, 32), dtype=float32)
Tensor("query_model_1/user_model_2/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("query_model_1/user_model_2/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("user_model_2/concat/concat:0", shape=(None, 32), dtype=float32)
Tensor("user_model_2/concat/concat:0", shape=(None, 32), dtype=float32)


INFO:tensorflow:Assets written to: /home/spark/ylv/workplace/model/assets


INFO:tensorflow:Assets written to: /home/spark/ylv/workplace/model/assets


In [18]:
load_model = tf.saved_model.load(path)

In [19]:
load_model({"user_id": np.array(["User_Fresher_NodeJS_Developer_DN_15"])})

(<tf.Tensor: shape=(1, 20), dtype=float32, numpy=
 array([[-16.24477 , -16.627834, -16.94488 , -17.620596, -17.843708,
         -17.948917, -18.210773, -18.481773, -18.576149, -18.633797,
         -18.810991, -18.836975, -18.91215 , -18.958263, -18.967714,
         -18.987963, -19.25468 , -19.46134 , -19.517828, -19.542816]],
       dtype=float32)>,
 <tf.Tensor: shape=(1, 20), dtype=string, numpy=
 array([[b'Job_Fresher_NodeJS_Developer_DN_Fresher_GBKWPB',
         b'Job_Fresher_NodeJS_Developer_DN_Fresher_XQDV2R',
         b'Job_Fresher_NodeJS_Developer_other_Fresher_PJZZPQ',
         b'Job_Fresher_Fullstack_Developer_other_Fresher_WM5YTT',
         b'Job_Fresher_NodeJS_Developer_other_Fresher_8T0RDK',
         b'Job_Junior_NodeJS_Developer_DN_Junior_2QWO3K',
         b'Job_Fresher_NodeJS_Developer_other_Fresher_RH8E05',
         b'Job_Fresher_NodeJS_Developer_HN_Fresher_RNJIF0',
         b'Job_Fresher_NodeJS_Developer_other_Fresher_B1F3LE',
         b'Job_Fresher_NodeJS_Developer_oth